In [173]:
import numpy as np
import sympy as sym
import control as ctrl
from matplotlib import pyplot as plt

setup 4
static friction 2.4
dynamic friction 2.4

In [174]:
# Define the  physical constants for the pendulum experiment

g=9.81  # m/s^2
L=0.125 # distance from pendulum's centre of mass to pivot in m
a=0.016 # radius of pulley in m
m=0.32  # mass of pendulum in kg
M=0.7  # mass of carriage in kg
I=8e-5  # moment of inertia on motor shaft in kg m^2
km = 0.08  # torque motor constant in Nm/A
ka = -0.50 # amplifier constant in A/V

gamma =  M/m + I/(m*np.power(a,2))

#scale factors to get physical units metres radians and seconds.
#in the crane/down position
# [x xdot Ltheta Lthetadot]=[CP CV PP PV]*Sc where,
Sc=np.diag([-1/12.5, -1/2.23, L/3.18, L/0.64])

# in the inverted position
# [x xdot Lphi Lphidot]=[CP CV PP PV]*Sp where,
Sp=np.diag([-1/12.5, -1/2.23, L/3.18, -L/0.64])

#controller amplifier gains on each measurement
opamp_c = np.diag([-20,-30, 20, -10]) # for crane controller
opamp_p = np.diag([10,20,30,-20]) # for inverted pendulum controller

#maximum torque from the motor in Nm
Tmax=0.4

# squares of the natural frequencies
om12=g/L 
om02=om12*(1+1/gamma)

#linearized crane  model

Ac=np.array([[0, 1, 0, 0,], [0, 0, om12-om02, 0], [0, 0, 0, 1], [0, 0, -om02, 0]])
Cc=-(ka*km/(m*a*gamma))*np.linalg.solve(Sc,opamp_c)
B=np.array([[0],[1],[0],[1]])

#linearized inverted pendulum model
Ap=np.array([[0, 1, 0, 0], [0, 0, om02-om12, 0], [0, 0, 0, 1], [0, 0, om02, 0]])
Cp=-(ka*km/(m*a*gamma))*np.linalg.solve(Sp,opamp_p)

# Sampling period 
h=0.0025 #sampling period
h_sim=0.002 #simulation step size

In [175]:
# build ss model using control

sys_c = ctrl.ss(Ac,B,Cc,0)

Q = np.diag([1, 0.01, 0.1, 0.01])
R = 0.1

K, S, E = ctrl.lqr(sys_c, Q, R)

# normalize the gains
K = K/np.max(np.abs(K))

print(K)

[[ 1.          0.91769184 -0.00723565 -0.11278234]]


Inverted pendulum fastest response time
The state velocities and input are not limited, and so the LQR controller aims to only minimize the displacements

data: 30

In [176]:
sys_p = ctrl.ss(Ap,B,Cp,0)

max_x = 0.4 # meters
max_theta = 60 * np.pi / 180 # degrees
max_xdot = 100 # m/s
max_thetadot = 100 # rad/s

max_input_power = 900 # W
max_torque = max_input_power / max_thetadot # Nm

max_u = max_torque / (a * gamma)

Q = np.diag([
    1/max_x**2, 
    1/max_xdot**2, 
    1/(L*max_theta)**2, 
    1/(L*max_thetadot)**2])

R = 1/max_u**2

print(R)

Kp, Sp, Ep = ctrl.lqr(sys_p, Q, R)


Pp = Kp @ np.linalg.inv(Cp)

if np.any(np.abs(Pp) > 1):
    Pp /= np.max(np.abs(Pp))
    print("warning : gains are scaled")

print("Gains : ")
print(Pp)

print("Poles : ")
print(Ep)

3.1640625e-05
Gains : 
[[0.97082743 1.         0.87961979 0.60904836]]
Poles : 
[-28.54015371+24.82060451j -28.54015371-24.82060451j
  -3.99393105 +2.90341968j  -3.99393105 -2.90341968j]


# 32 power 300
Gains : 
[[0.48       0.53731688 0.54687807 0.40124991]]
Poles : 
[-17.19464326+13.71349119j -17.19464326-13.71349119j
  -3.9456746  +2.90995092j  -3.9456746  -2.90995092j]

# 31 power 500
Gains : 
[[0.8        0.855881   0.80390787 0.5744239 ]]
Poles : 
[-21.51191845+18.30786104j -21.51191845-18.30786104j
  -3.98032181 +2.90547436j  -3.98032181 -2.90547436j]

# 34 power 700
Gains : 
[[0.95669605 1.         0.90193571 0.63290026]]
Poles : 
[-25.2181614 +21.86370406j -25.2181614 -21.86370406j
  -3.98995595 +2.90403728j  -3.98995595 -2.90403728j]

# 35 power 900
Gains : 
[[0.97082743 1.         0.87961979 0.60904836]]
Poles : 
[-28.54015371+24.82060451j -28.54015371-24.82060451j
  -3.99393105 +2.90341968j  -3.99393105 -2.90341968j]

Optimal control

In [195]:

sys_p = ctrl.ss(Ap,B,Cp,0)

max_x = 0.4 # meters
max_theta = 60 * np.pi / 180 # degrees
max_xdot = 5 # m/s
max_thetadot = 5 # rad/s

max_input_power = 7
max_torque = max_input_power / max_thetadot # Nm
max_u = max_torque / (a * gamma)

Q = np.diag([
    1/max_x**2, 
    1/max_xdot**2, 
    1/(L*max_theta)**2, 
    1/(L*max_thetadot)**2])

R = 1/max_u**2
print(R)

Kp, Sp, Ep = ctrl.lqr(sys_p, Q, R)


Pp = Kp @ np.linalg.inv(Cp)

if np.any(np.abs(Pp) > 1):
    Pp /= np.max(np.abs(Pp))
    print("warning : gains are scaled")

print("Gains : ")
print(Pp)

print("Poles : ")
print(Ep)

0.0013075972576530616
Gains : 
[[0.224      0.31892512 0.41711284 0.37027587]]
Poles : 
[-46.53147974+0.j          -3.22625738+3.27620183j
  -3.22625738-3.27620183j  -5.51523777+0.j        ]


# 36 power 10
[[0.32       0.44649708 0.55137318 0.49925406]]
Poles : 
[-65.08420352+0.j          -5.43796833+0.j
  -3.26958823+3.34815994j  -3.26958823-3.34815994j]

# 38 power 15
Gains : 
[[0.48       0.66003764 0.77765067 0.71646929]]
Poles : 
[-96.48339016+0.j          -3.29331246+3.38907627j
  -3.29331246-3.38907627j  -5.39610023+0.j        ]

# 39 power 17
Gains : 
[[0.544      0.74560721 0.86855847 0.80370474]]
Poles : 
[-109.11570714+0.j           -3.2975587 +3.39651778j
   -3.2975587 -3.39651778j   -5.38864244+0.j        ]

This does seem to work

## Changing max x

# 37 max = 0.1, power = 7
Gains : 
[[0.896      0.87652021 0.75406732 0.64043166]]
Poles : 
[-46.19050811+0.j          -6.24238354+5.55814879j
  -6.24238354-5.55814879j  -6.72571361+0.j        ]

# 40 max = 0.2, power = 7
Gains : 
[[0.448      0.51868237 0.54153398 0.46819955]]
Poles : 
[-46.46423563+0.j          -4.36405288+4.40951646j
  -4.36405288-4.40951646j  -6.06790691+0.j        ]

# 41 max = 0.4, power = 7
Gains : 
[[0.224      0.31892512 0.41711284 0.37027587]]
Poles : 
[-46.53147974+0.j          -3.22625738+3.27620183j
  -3.22625738-3.27620183j  -5.51523777+0.j        ]

Clearly overshoot is increasing with max x

In [205]:


# generic equally weighted cost function
Q = np.eye(4)
R = 0.0005

Kp, Sp, Ep = ctrl.lqr(sys_p, Q, R)

Pp = Kp @ np.linalg.inv(Cp)

print(Pp)
print(Ep)


[[0.1448972  0.51396675 0.61519804 0.53601152]]
[-64.24720178+0.j          -1.00043628+0.j
  -6.83872761+2.79938288j  -6.83872761-2.79938288j]


# 41
[[0.1448972  0.51396675 0.61519804 0.53601152]]
[-64.24720178+0.j          -1.00043628+0.j
  -6.83872761+2.79938288j  -6.83872761-2.79938288j]
